<div align="center">

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.png)](https://colab.research.google.com/github/wisaaco/AA_DistributedSystems_Lab/blob/main/U9-Spark/SparkExamples.ipynb)

Si no funciona el botó podeu copiar el següent [enllaç](https://colab.research.google.com/github/wisaaco/AA_DistributedSystems_Lab/blob/main/U9-Spark/SparkExamples.ipynb)

</div>

Referències:
- https://spark.apache.org/docs/latest/rdd-programming-guide.html

In [1]:
!pip install pyspark


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")conf.close()

In [4]:
sc = SparkContext.getOrCreate(conf=conf)

25/03/24 17:33:37 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.ja

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:334)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [5]:
sc.close()

NameError: name 'sc' is not defined

In [118]:
# data = sc.textFile("sample_data/README.md")
data = sc.textFile("sample_data/README.md")

In [119]:
type(data)

pyspark.rdd.RDD

In [63]:
len(data)

TypeError: object of type 'RDD' has no len()

In [64]:
for line in data:
    print(line)

TypeError: 'RDD' object is not iterable

In [65]:
for line in data.collect():
  print(line)

This directory includes a few sample datasets to get you started.

*   `california_housing_data*.csv` is California housing data from the 1990 US
    Census; more information is available at:
    https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub

*   `mnist_*.csv` is a small sample of the
    [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), which is
    described at: http://yann.lecun.com/exdb/mnist/

*   `anscombe.json` contains a copy of
    [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet); it
    was originally described in

    Anscombe, F. J. (1973). 'Graphs in Statistical Analysis'. American
    Statistician. 27 (1): 17-21. JSTOR 2682899.

    and our copy was prepared by the
    [vega_datasets library](https://github.com/altair-viz/vega_datasets/blob/4f67bdaad10f45e3549984e17e1b3088c731503d/vega_datasets/_data/anscombe.json).


In [67]:
data2 = sc.parallelize(["pandas", "i like pandas"])
type(data2)

pyspark.rdd.RDD

In [68]:
len(data2)

TypeError: object of type 'RDD' has no len()

# RDD
A **Resilient Distributed Dataset (RDD)**, the basic abstraction in Spark.

https://spark.apache.org/docs/1.5.1/api/python/pyspark.html#pyspark.RDD

## Operations
- **Actions** are operations that give non-RDD values. The values of action are stored to drivers or to the external storage system. It brings laziness of RDD into motion.<br/>
```python
lines.count() -> int
```

- **Transformations** are functions that produces new RDD from the existing RDDs. It takes RDD as input and produces one or more RDD as output. They are operations on RDDs that return a new RDD. As discussed in “Lazy Evaluation” , transformed RDDs are computed lazily, only when you use them in an action. Many transformations are element-wise; that is, they work on one element at a time; but this is not true for all transformations.

**Transformations** : Filter()

In [69]:
inputRDD = sc.textFile("sample_data/README.md")
samplesRDD = inputRDD.filter(lambda x: "sample" in x)
type(samplesRDD)

pyspark.rdd.PipelinedRDD

## Actions
They are the operations that return a final value to the driver program or write data to an external storage system. Actions force the evaluation of the transformations required for the RDD they were called on, since they need to actually produce output

- collect()
- count()
- countByValue()
- take(num)
- top(num)
- takeOrdered(num)(ordering)
- takeSample(...)
- reduce()
- fold
- aggregate
- foreach


In [71]:
for line in inputRDD.collect():
  print(line)
  break

This directory includes a few sample datasets to get you started.


In [72]:
print("Total Input: %i "%samplesRDD.count())


Total Input: 2 


In [77]:
for line in samplesRDD.take(3):
  print(line)

This directory includes a few sample datasets to get you started.
*   `mnist_*.csv` is a small sample of the


In [78]:
samplesRDD.top(2)

['This directory includes a few sample datasets to get you started.',
 '*   `mnist_*.csv` is a small sample of the']

In [55]:
nums = sc.parallelize([1, 2, 2, 2])
nums.countByValue()

defaultdict(int, {1: 1, 2: 3})

**reduce()**, which takes a function that operates on two elements of the type in your RDD and returns a new element of the same type.

In [79]:
nums = sc.parallelize([1, 2, 3, 4])
sum = nums.reduce(lambda x, y: x + y)
print(sum)

10


takes a neutral “zero value” to be used for the initial call on each partition.

In [ ]:
nums = sc.parallelize([1, 2, 3, 4])
sum = nums.fold(1,lambda x, y: x + y)
print(sum)

12


aggregate() function frees us from the constraint of having the return be the same type as the RDD we are working on.<br/>
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.aggregate.html

In [95]:
nums = sc.parallelize([1, 2, 3, 4])
seqOp = lambda acc, num: (acc[0] + num, acc[1] + 1) # a function used to accumulate results within a partition
combOp = lambda partition_1, partition_2: (partition_1[0] + partition_2[0], partition_1[1] + partition_2[1]) #an associative function used to combine results from different partitions
sumCount = nums.aggregate((0, 0),seqOp,combOp)

print(type(sumCount))
print(len(sumCount))
print(sumCount)

sc.parallelize([]).aggregate((1, 0), seqOp, combOp)

<class 'tuple'>
2
(10, 4)


(2, 0)

 We could have:
- partition 1: [1,2] 
- partition 2: [3,4]

SeqOp:
- partition 1: (3,2) 
- partition 2: (7,2)

Combop: (10,4)

Reflexión

In [98]:
rdd = sc.parallelize([1, 2, 3, 4, 5 ,6, 7, 8, 9, 10], 3) #3 partitions
rdd.glom().collect() #Glom. Return an RDD created by coalescing all elements within each partition into a list.


[[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]]

In [105]:
seqOp = lambda acc, num: (acc[0] * num, acc[1] + 1) 
combOp = lambda partition_1, partition_2: (partition_1[0] + partition_2[0], partition_1[1] + partition_2[1]) #an associative function used to combine results from different partitions
unk_op = rdd.aggregate(
  (0, -1), seqOp, combOp
)
# result = ?

In [106]:
unk_op

(0, 6)

## Transformations
Narrow Transformation:
- map
- flatMap
- MapParition
- Filter
- Sample
- Union

In [108]:
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x)
print(squared.collect())


[1, 4, 9, 16]


In [121]:
lines = sc.parallelize(["hello world", "hi Pepe, how are you?"])
words = lines.flatMap(lambda line: line.split(" "))
print(words.collect())


['hello', 'world', 'hi', 'Pepe,', 'how', 'are', 'you?']


In [123]:
# Return a new RDD by applying a function to each partition of this RDD.
rdd = sc.parallelize([(1, 2), (3, 4), (5, 6), (7, 8)], 2)
print(rdd.glom().collect() )
def average_partition(iterator):
     x_sum = 0
     y_sum = 0
     count = 0
     for (x, y) in iterator:
         x_sum += x
         y_sum += y
         count += 1
     yield (x_sum/count, y_sum/count)

avg_rdd = rdd.mapPartitions(average_partition)
avg_rdd.collect()


[[(1, 2), (3, 4)], [(5, 6), (7, 8)]]


[(2.0, 3.0), (6.0, 7.0)]

In [127]:
lines = sc.parallelize(["hello world", "hi Pepe, who are you?"])
lines.filter(lambda x: "wo" in x).collect()


['hello world']

In [132]:
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sample.html
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.sample(False,0.5,seed=1).collect()


[1, 3, 7, 10]

Union(), disctint(), intersection(), subtract(), cartesian()

In [133]:
lines = sc.parallelize(["hello world", "hi Pepe, who are you?"])
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
uRDD = rdd.union(lines)
uRDD.collect()


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'hello world', 'hi Pepe, who are you?']

# **Pair RDDs**

Pair RDDs are a useful building block in many programs, as they expose operations that allow you to act on each key in parallel or regroup data across the network. For example, pair RDDs have a reduceByKey() method that can aggregate data separately for each key, and a join() method that can merge two RDDs together by grouping elements with the same key.


Wide transformation:

- keys()
- values()
- reduceByKey(func)
- groupByKey()
- combineByKey(...)
- mapValues(func)
- flatMapValues(func)
- sortByKey()
- countByKey()
- collectAsMap()
- lookup(key)



In [138]:
lines = sc.textFile("sample_data/README.md")


In [139]:
pairs = lines.map(lambda x: (x.split(" ")[0], x))
pairs.take(1)

[('This', 'This directory includes a few sample datasets to get you started.')]

In [140]:
pairs.keys().count()

19

In [141]:
pairs.collect()

[('This', 'This directory includes a few sample datasets to get you started.'),
 ('', ''),
 ('*',
  '*   `california_housing_data*.csv` is California housing data from the 1990 US'),
 ('', '    Census; more information is available at:'),
 ('',
  '    https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub'),
 ('', ''),
 ('*', '*   `mnist_*.csv` is a small sample of the'),
 ('',
  '    [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), which is'),
 ('', '    described at: http://yann.lecun.com/exdb/mnist/'),
 ('', ''),
 ('*', '*   `anscombe.json` contains a copy of'),
 ('',
  "    [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet); it"),
 ('', '    was originally described in'),
 ('', ''),
 ('',
  "    Anscombe, F. J. (1973). 'Graphs in Statistical Analysis'. American"),
 ('', '    Statistician. 27 (1): 17-21. JSTOR 2682899.'),
 ('', ''),
 ('', '    and our copy was prepared by th

In [142]:
pairs.keys().collect()

['This',
 '',
 '*',
 '',
 '',
 '',
 '*',
 '',
 '',
 '',
 '*',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [47]:
pairs.keys().distinct().count()

3

In [ ]:
## Reflexion
words = lines.flatMap(lambda x: x.split(" "))
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [49]:
for kv in result.sortByKey().collect():
  print(kv)

('', 51)
("'Graphs", 1)
('(1):', 1)
('(1973).', 1)
('*', 3)
('17-21.', 1)
('1990', 1)
('2682899.', 1)
('27', 1)
('American', 1)
("Analysis'.", 1)
('Anscombe,', 1)
('California', 1)
('Census;', 1)
('F.', 1)
('J.', 1)
('JSTOR', 1)
('Statistical', 1)
('Statistician.', 1)
('This', 1)
('US', 1)
("[Anscombe's", 1)
('[MNIST', 1)
('[vega_datasets', 1)
('`anscombe.json`', 1)
('`california_housing_data*.csv`', 1)
('`mnist_*.csv`', 1)
('a', 3)
('and', 1)
('at:', 2)
('available', 1)
('by', 1)
('contains', 1)
('copy', 2)
('data', 1)
('database](https://en.wikipedia.org/wiki/MNIST_database),', 1)
('datasets', 1)
('described', 2)
('directory', 1)
('few', 1)
('from', 1)
('get', 1)
('housing', 1)
('http://yann.lecun.com/exdb/mnist/', 1)
('https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub', 1)
('in', 2)
('includes', 1)
('information', 1)
('is', 4)
('it', 1)
('library](https://github.com/altair-viz/vega_datasets/blob/4f67bdaad1

**Join** operator is an inner join.

In [146]:
data1 = [("a", 3), ("b", 4), ("a", 1)]
data2 = [("a", 5), ("b", 1), ("c", 1)]
d1 = sc.parallelize(data1)
d2 = sc.parallelize(data2)

In [147]:
for kv in d1.join(d2).collect():
  print(kv)

('b', (4, 1))
('a', (3, 5))
('a', (1, 5))


leftOuterJoin(other) ,  rightOuterJoin(other)

In [148]:
for kv in d1.leftOuterJoin(d2).collect():
  print(kv)

('b', (4, 1))
('a', (3, 5))
('a', (1, 5))


In [149]:
for kv in d1.rightOuterJoin(d2).collect():
  print(kv)

('b', (4, 1))
('c', (None, 1))
('a', (3, 5))
('a', (1, 5))


**Lookup**

In [159]:
l = range(100)
l1 = range(1,101)
rdd = sc.parallelize(zip(l, l1), 10)
print(rdd.take(5))
rdd.lookup(42)  # slow




[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]


[43]

In [160]:
sorted = rdd.sortByKey()
sorted.lookup(42)  # fast


[43]

In [171]:
import os 
os.environ["PYTHONHASHSEED"]="1" #Spark sessions: coherencia entre tipos 

rdd2 = sc.parallelize([(('a', 'b'), 'c')]).groupByKey()
list(rdd2.lookup(('a', 'b'))[0])

['c']

# Ejemplo 

$$PageRank(A) = \frac{(1 - d)}{N} + d * \sum_{B\in in(A)} \frac{PageRank(B)}{L(B)}$$


Donde:

- A y B son páginas
- `PageRank(A)` es el valor de PageRank para la página A.
- `d` es el factor de amortiguación (generalmente se establece en 0.85 en la práctica).
- `N` es el número total de páginas en la red.
- `Σ` representa la suma sobre todas las páginas B que enlazan a la página A.
- in(A) es el conjunto de páginas que enlazan a la página A.
- `PageRank(B)` es el valor de PageRank de la página B.
- `L(B)` es el número de enlaces salientes desde la página B.


Supongamos que tenemos cuatro páginas web (A, B, C y D) en una red y que inicialmente todas tienen un PageRank igual. El factor de amortiguación (d) es 0.85.

Relaciones:

- A <- B
- B <- A, C
- C <- B
- D <- B

Iteraciones:

* Iteración 0 (valores iniciales):



PageRank(A) = PageRank(B) = PageRank(C) = PageRank(D) = 0.25

* Iteración 1:


\begin{align*}
PageRank(A) & = \frac{(1 - 0.85)}{4} + 0.85 \cdot \frac{PageRank(B)}{1} \\
& = 0.0375 + 0.85 \cdot 0.25 = 0.2875
\end{align*}

\begin{align*}
PageRank(B) & = \frac{(1 - 0.85)}{4} + 0.85 \cdot \left(\frac{PageRank(A)}{1} + \frac{PageRank(C)}{1}\right) \\
& = 0.0375 + 0.85 \cdot (0.2875 + 0.25) = 0.675
\end{align*}

\begin{align*}
PageRank(C) & = \frac{(1 - 0.85)}{4} + 0.85 \cdot \frac{PageRank(B)}{1} \\
& = 0.0375 + 0.85 \cdot 0.675 = 0.6025
\end{align*}


\begin{align*}
PageRank(D) & = \frac{(1 - 0.85)}{4} + 0.85 \cdot \frac{PageRank(B)}{1} \\
& = 0.0375 + 0.85 \cdot 0.675 = 0.6025
\end{align*}



In [ ]:
# REF: https://github.com/Proxy08/PageRank/blob/main/pagerank-spark.py

In [2]:
import re

def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [1]:
from pyspark.sql import SparkSession


spark = SparkSession\
    .builder\
    .appName("PageRank")\
    .getOrCreate()

lines = spark.read.text("pageRank_data.txt")
lines = lines.rdd.map(lambda r: r[0])
for i in lines.collect():
  print(i)

25/03/24 17:33:00 WARN Utils: Your hostname, MacBook-Pro-de-Isaac-2.local resolves to a loopback address: 127.0.0.1; using 10.162.8.240 instead (on interface en0)
25/03/24 17:33:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 17:33:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:334)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [ ]:
lines = lines.rdd.map(lambda r: r[0])

In [ ]:
for i in lines.collect():
  print(i)

In [ ]:

# Loads all URLs from input file and initialize their neighbors.
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()


In [ ]:
for i in links.collect():
  print(i[0])
  for j in i[1]:
    print("\t",j)

In [ ]:

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [ ]:
for i in ranks.collect():
  print(i[0],i[1])


In [ ]:
t = links.join(ranks)
t.take(1)

In [ ]:
contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
    url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
))

In [ ]:
a = links.join(ranks).flatMap(lambda url_urls_rank: (url_urls_rank[1][0], url_urls_rank[1][1]))


In [ ]:
for i in a.collect():
 if (type(i)!=float):
  for x in i:
    print(x)
 else:
  print(i)


In [ ]:
contribs.take(1)

In [ ]:
for i in contribs.collect():
  print(i)

In [ ]:
ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
for i in ranks.collect():
  print(i)

In [ ]:
from operator import add

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(5):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
        url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
    ))

    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

# Collects all URL ranks and dump them to console.
for (link, rank) in ranks.collect():
    print("%s has rank: %s." % (link, rank))



## Actividad

In [ ]:
!pip install faker faker-commerce

Tenemos un archivo de transacciones
```csv
order_id, product_id, quantity, price, status
1, 101, 2, 15.99, COMPLETE
2, 103, 1, 29.99, PENDING
3, 101, 1, 15.99, CANCELED
...
```

llamado `orders.csv`

In [183]:
import pandas as pd
import numpy as np
np.random.seed(1)
size = 1000
states = ["COMPLETE","PENDING","CANCELED"]
product_id = np.random.choice(np.random.randint(200,10090,size=500),size)
quantity = np.random.randint(1,10,size)
price = np.random.randint(1,3000,size)
status = np.random.choice(states,size,p=[0.7,0.25,0.05])
pd.DataFrame({"order_id":range(size),"product_id":product_id, "quantity":quantity, "price":price, "status":status}).to_csv("orders.csv")

Y un archivo de productos, llamado `products.csv`:

```csv
product_id, product_name, category, company
101, Widget A, Electronics, Leach-Smith
103, Widget B, Home, Hopkins Inc
...
```

In [208]:
from faker import Faker
import faker_commerce
Faker.seed(1)

u_product= np.unique(product_id)
size = len(u_product)
print(size)

# category= ["Automative","Home","Electronics","Retail","Health Care","Books","Media"]
fake = Faker()
fake.add_provider(faker_commerce.Provider)
    
pd.DataFrame({"product_id":u_product, 
              "product_name":[fake.name() for _ in range(size)], 
              "category":[fake.ecommerce_category() for _ in range(size)], 
              "company":[fake.company() for _ in range(size)], 
              }).to_csv("products.csv")

418


## Tareas
- A. Combinar ambos ficheros 
- B. Controlar la facturación:
  - a. Contar la cantidad de pedidos en cada estado (COMPLETE, CANCELED, PENDING).
  - b. Total de Ingresos por Producto (Filtrar solo los pedidos en estado COMPLETE) y calcular el ingreso total para cada product_id multiplicando quantity por price. Mapea los resultados a una estructura con el nombre del producto y la categoría.
  

In [ ]:
# TODO